In [13]:
def evaluate_accuracy(data_iter, net1, net2, device):
#     if device is None and isinstance(net1, torch.nn.Module):
#         # 如果没指定device就使用net的device
#         device = list(net1.parameters())[0].device 
    acc_sum, n = 0.0, 0
    net1.eval()
    net2.eval()
    with torch.no_grad():
        for X, y in data_iter:
#             net1.eval() # 评估模式, 这会关闭dropout
            preds1 = net1(X.to(device))
            preds2 = net2(X.to(device))
            preds = 0 * softmax(preds1.cpu().numpy()) + 1 * softmax(preds2.cpu().numpy())
            
            acc_sum += (torch.from_numpy(preds).to(device).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            
#             acc_sum += (net1(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
#             net.train() # 改回训练模式
            n += y.shape[0]
    return acc_sum / n

In [9]:
def evaluate_accuracy2(data_iter, net, device):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
           
            net.eval() # 评估模式, 这会关闭dropout
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            net.train() # 改回训练模式
            n += y.shape[0]
    return acc_sum / n

In [21]:
evaluate_accuracy2(test_iter_152, resnet152, device)

0.9135104578858112

In [23]:
%matplotlib inline
import torch
import time
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models
import os
# from scipy.special import softmax
from torch.nn.functional import softmax

import sys
sys.path.append("..") 
# import d2lzh_pytorch as d2l

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [24]:
def load_data_face_18(batch_size):
    
    transform = torchvision.transforms.Compose([
        #torchvision.transforms.Grayscale(num_output_channels=3), # 彩色图像转灰度图像num_output_channels默认1
        #torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.Resize([224, 224]),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std= [0.229, 0.224, 0.225])

        ])
    
    test_imgs = torchvision.datasets.ImageFolder('dataset/test', transform=transform)
    test_iter = torch.utils.data.DataLoader(test_imgs, batch_size=batch_size, shuffle=False, num_workers=4)

    
    return test_iter

In [25]:
def load_data_face_50(batch_size):
    transform = torchvision.transforms.Compose([
       # torchvision.transforms.Grayscale(num_output_channels=1), # 彩色图像转灰度图像num_output_channels默认1
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.Resize([330,330]),
        torchvision.transforms.CenterCrop([224, 224]),
        
        torchvision.transforms.ToTensor()
    ])

    test_imgs = torchvision.datasets.ImageFolder('dataset/test', transform=transform)

    test_iter = torch.utils.data.DataLoader(test_imgs, batch_size=batch_size, shuffle=False, num_workers=4)

    return test_iter

In [26]:
def load_data_face_152(batch_size):
    
    transform = torchvision.transforms.Compose([
       # torchvision.transforms.Grayscale(num_output_channels=1), # 彩色图像转灰度图像num_output_channels默认1
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.Resize([330,330]),
        torchvision.transforms.CenterCrop([224, 224]),
        
        torchvision.transforms.ToTensor()
    ])
    
    test_imgs = torchvision.datasets.ImageFolder('dataset/test', transform=transform)
    test_iter = torch.utils.data.DataLoader(test_imgs, batch_size=batch_size, shuffle=False, num_workers=4)

    
    return test_iter

In [27]:
resnet18 = torch.load('./model_resnet18-925.pkl').to(device)

In [28]:
resnet50 = torch.load('./resnet50(923).pkl').to(device)

In [29]:
resnet152 = torch.load('./work/resnet152.pkl').to(device)

In [30]:
batch_size = 32
test_iter_18 = load_data_face_18(batch_size) 
test_iter_50 = load_data_face_50(batch_size)
test_iter_152 = load_data_face_152(batch_size)

In [37]:
def evaluate_accuracy_two(data_iter_18, data_iter_50, net1, net2, device):
#     if device is None and isinstance(net1, torch.nn.Module):
#         # 如果没指定device就使用net的device
#         device = list(net1.parameters())[0].device 
    acc_sum, n = 0.0, 0
    net1.eval()
    net2.eval()
    preds_18 = []
#     preds_50 = []
    with torch.no_grad():
        for X1, y1 in data_iter_18:
#             net1.eval() # 评估模式, 这会关闭dropout
            preds18 = net1(X1.to(device))
#             preds2 = net2(X.to(device))
#             preds18 = softmax(preds18.cpu().numpy())
            preds18 = softmax(preds18,dim=1)
#             preds = 0 * softmax(preds1.cpu().numpy()) + 1 * softmax(preds2.cpu().numpy())
            preds_18.append(preds18)
      
        m = 0
        for X2, y2 in data_iter_50:
            preds50 = net2(X2.to(device))
#             preds2 = net2(X.to(device))
            preds50 = softmax(preds50,dim=1)
#             preds50 = softmax(preds50.cpu().numpy())
            preds =2/5 * preds50 + 2/5 * preds_18[m]
#             preds = 2/5 * preds152 + 1/5 * preds_18[m] + 2/5 * preds_50[m]
            acc_sum += (preds.argmax(dim=1) == y2.to(device)).float().sum().cpu().item()
            m += 1
#             acc_sum += (net1(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
#             net.train() # 改回训练模式
            n += y2.shape[0]
    return acc_sum / n

In [39]:
evaluate_accuracy_two(test_iter_50, test_iter_152, resnet50, resnet152, device)

0.9366873940079141

In [44]:
def evaluate_accuracy(data_iter_18, data_iter_50, data_iter_152, net1, net2, net3, device):
#     if device is None and isinstance(net1, torch.nn.Module):
#         # 如果没指定device就使用net的device
#         device = list(net1.parameters())[0].device 
    acc_sum, n = 0.0, 0
    net1.eval()
    net2.eval()
    net3.eval()
    preds_18 = []
    preds_50 = []
    with torch.no_grad():
        for X1, y1 in data_iter_18:
#             net1.eval() # 评估模式, 这会关闭dropout
            preds18 = net1(X1.to(device))
#             preds2 = net2(X.to(device))
            preds18 = softmax(preds18,dim=1)
#             preds = 0 * softmax(preds1.cpu().numpy()) + 1 * softmax(preds2.cpu().numpy())
            preds_18.append(preds18)
        for X2, y2 in data_iter_50:
#             net1.eval() # 评估模式, 这会关闭dropout
            preds50 = net2(X2.to(device))
#             preds2 = net2(X.to(device))
            preds50 = softmax(preds50,dim=1)
#             preds = 0 * softmax(preds1.cpu().numpy()) + 1 * softmax(preds2.cpu().numpy())
            preds_50.append(preds50)
        m = 0
        for X3, y3 in data_iter_152:
            preds152 = net3(X3.to(device))
#             preds2 = net2(X.to(device))
            preds152 = softmax(preds152,dim=1)
#             preds =1/5 * preds152 + 2/5 * preds_50[m]
            preds = 3/8 * preds152 + 0 * preds_18[m] + 3/8 * preds_50[m]
            acc_sum += (preds.argmax(dim=1) == y3.to(device)).float().sum().cpu().item()
            m += 1
#             acc_sum += (net1(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
#             net.train() # 改回训练模式
            n += y3.shape[0]
    return acc_sum / n

In [46]:
test_acc = evaluate_accuracy(test_iter_18, test_iter_50, test_iter_152, resnet18, resnet50, resnet152, device)
print(test_acc)

0.9389485585076315


In [44]:
from scipy.special import softmax

In [14]:
!pip list

Package                Version   
---------------------- ----------
absl-py                0.7.0     
alembic                0.9.9     
asn1crypto             0.24.0    
astor                  0.7.1     
async-generator        1.10      
attrs                  18.2.0    
backcall               0.1.0     
beautifulsoup4         4.6.3     
bleach                 1.5.0     
bokeh                  0.13.0    
certifi                2018.11.29
cffi                   1.11.5    
chardet                3.0.4     
cloudpickle            0.5.6     
cnn-finetune           0.3       
conda                  4.6.2     
cryptography           2.5       
cycler                 0.10.0    
Cython                 0.28.5    
cytoolz                0.9.0.1   
d2lzh                  1.0.0     
dask                   1.1.1     
decorator              4.3.0     
dill                   0.2.9     
entrypoints            0.3       
fastcache              1.0.2     
gast                   0.2.2     
gluonbook     